A tool to noodle around with importing hex codes to ArcMap Colors. We start with our imports

In [169]:
import os
import shutil
import pandas as pd
import sqlite3 as db
import webcolors as wc
import sqlalchemy


In [170]:
#os.remove('WFRC.stylx')
shutil.copy("WFRC copy.stylx","WFRC.stylx")

'WFRC.stylx'

To give an example of what the style DB looks like

In [171]:
eng = sqlalchemy.create_engine('sqlite:///WFRC.stylx')
#print(eng)
#eng.connect
with eng.connect() as conn:
    df = pd.read_sql('select * from ITEMS', conn)
df

,ID,CLASS,CATEGORY,NAME,TAGS,CONTENT,KEY
0,1,1,,WFRC Blue,rgb;black,"{""type"":""CIMRGBColor"",""values"":[2,60,91,100]} ",wfrc_logo


From there, we pull in the CSV file containing the colors

In [172]:
style_colors = pd.read_csv("Color Inventory.csv")
style_colors

,Sort Order,Secondary Order,Name,Color,Hex,RGB,Color Category,Color Type,CMYK,Color Origin
0,1,1,WFRC Blue,NaN,#023c5b,"rgb(2,60,91)",Blue,Logo Brights,C 100% M 75.19% Y 40.82% K 30.87%,WFRC Mountains Logo
1,2,2,Blue,NaN,#036fa8,"rgb(3,111,168)",Blue,Logo Brights,C 90.21% M 52.89% Y 11.12% K .48%,SLawless
2,3,3,Purple,NaN,#553c8f,"rgb(85,60,143)",Purple,Logo Brights,C 81.88% M 91.64% Y 5.89% K .63%,SLawless
3,4,4,Red,NaN,#d32f00,"rgb(211,47,0)",Red,Logo Brights,C 11.11% M 94.28% Y 100% K 2.84%,SLawless
4,5,5,Orange,NaN,#ea7b00,"rgb(234,123,0)",Orange,Logo Brights,C 5.08% M 62.17% Y 100% K .15%,SLawless
5,6,6,Yellow,NaN,#eab500,"rgb(234,181,0)",Yellow,Logo Brights,C 8.96% M 28.46% Y 100% K 0%,SLawless
6,7,7,Green,NaN,#007426,"rgb(0,116,38)",Green,Logo Brights,C 88.8% M 29.32% Y 100% K 18.94%,SLawless
7,8,1,RTP Fuschia,NaN,#9d4879,"rgb(157,72,121)",Purple,RTP Icons,C 39.86% M 84.49% Y 27.53% K 4.09%,2019-2050 RTP Infographics
8,9,2,RTP Maroon,NaN,#6f2d3d,"rgb(111,45,61)",Purple,RTP Icons,C 40.06% M 86.14% Y 58.46% K 38.83%,2019-2050 RTP Infographics
9,10,3,RTP Red,NaN,#c23c33,"rgb(194,60,51)",Red,RTP Icons,C 17.09% M 90.15% Y 88.75% K 6.32%,2019-2050 RTP Infographics


Why is Secondary Order prepended with a space??

In [173]:
style_colors.columns

Index(['Sort Order', ' Secondary Order', 'Name', 'Color', 'Hex', 'RGB',
       'Color Category', 'Color Type', 'CMYK', 'Color Origin'],
      dtype='object')

Create a list-based RGBA column, dispose of uncessarary columns (sorry this is all hardcoded. If your CSV file looks different, you'll have to rebuild the details.)

In [174]:
style_colors = style_colors.rename(columns={"Sort Order":"ID"})
style_colors["colCat"] = style_colors['Hex'].apply(wc.hex_to_rgb).apply(list)
for code in style_colors.colCat:
    code.append(100)
style_colors = style_colors.drop(columns=["CMYK", "Color", "RGB"," Secondary Order", "Hex"])
style_colors

,ID,Name,Color Category,Color Type,Color Origin,colCat
0,1,WFRC Blue,Blue,Logo Brights,WFRC Mountains Logo,"[2, 60, 91, 100]"
1,2,Blue,Blue,Logo Brights,SLawless,"[3, 111, 168, 100]"
2,3,Purple,Purple,Logo Brights,SLawless,"[85, 60, 143, 100]"
3,4,Red,Red,Logo Brights,SLawless,"[211, 47, 0, 100]"
4,5,Orange,Orange,Logo Brights,SLawless,"[234, 123, 0, 100]"
5,6,Yellow,Yellow,Logo Brights,SLawless,"[234, 181, 0, 100]"
6,7,Green,Green,Logo Brights,SLawless,"[0, 116, 38, 100]"
7,8,RTP Fuschia,Purple,RTP Icons,2019-2050 RTP Infographics,"[157, 72, 121, 100]"
8,9,RTP Maroon,Purple,RTP Icons,2019-2050 RTP Infographics,"[111, 45, 61, 100]"
9,10,RTP Red,Red,RTP Icons,2019-2050 RTP Infographics,"[194, 60, 51, 100]"


Set up the tagging system

In [175]:
existing_tags = ["Color Category", "Color Type", "Color Origin"]
new_tags = ["color", "colors"]
tags = new_tags + existing_tags
ntstr = ""
for t in new_tags: ntstr = ntstr + str(t) +";"
ntstr

'color;colors;'

Again, sorry for the kludge. I really wanted to use elementwise operations rather than having to iterate. If someone knows how to do the kind of operation shown below elementwise for arbitrary lengths of `existing_tag`, lmk!

In [176]:

style_colors["tags"] =  ntstr + style_colors[existing_tags[0]] + ";" + style_colors[existing_tags[1]] + ";" + style_colors[existing_tags[2]]
style_colors = style_colors.drop(columns=existing_tags)
style_colors

,ID,Name,colCat,tags
0,1,WFRC Blue,"[2, 60, 91, 100]",color;colors;Blue;Logo Brights;WFRC Mountains ...
1,2,Blue,"[3, 111, 168, 100]",color;colors;Blue;Logo Brights;SLawless
2,3,Purple,"[85, 60, 143, 100]",color;colors;Purple;Logo Brights;SLawless
3,4,Red,"[211, 47, 0, 100]",color;colors;Red;Logo Brights;SLawless
4,5,Orange,"[234, 123, 0, 100]",color;colors;Orange;Logo Brights;SLawless
5,6,Yellow,"[234, 181, 0, 100]",color;colors;Yellow;Logo Brights;SLawless
6,7,Green,"[0, 116, 38, 100]",color;colors;Green;Logo Brights;SLawless
7,8,RTP Fuschia,"[157, 72, 121, 100]",color;colors;Purple;RTP Icons;2019-2050 RTP In...
8,9,RTP Maroon,"[111, 45, 61, 100]",color;colors;Purple;RTP Icons;2019-2050 RTP In...
9,10,RTP Red,"[194, 60, 51, 100]",color;colors;Red;RTP Icons;2019-2050 RTP Infog...


Concatenate `colCat` to a new column called `content`

In [177]:
#A mistake in this line made my life a nightmare for FOUR HOURS!
style_colors["content1"]='{"type":"CIMRGBColor","values":' + style_colors["colCat"].apply(str).str.replace(" ","") + "}"

contentlist = []
#for r in style_colors["content"]:
#    contentlist.append(sqlalchemy.types.BLOB(r))
#print(contentlist)
#style_colors["content"]=contentlist
style_colors["content1"]=style_colors["content1"].astype(bytes)

#Possibly uncessary kludge, but I ain't fixing it
for r in style_colors["content1"]:
    contentlist.append(r + bytes(1))
style_colors["content"]=contentlist
#print(contentlist)
style_colors.insert(1, "Class", 1)
style_colors.insert(2, "Category", "")
style_colors

[b'{"type":"CIMRGBColor","values":[2,60,91,100]}\x00\x00', b'{"type":"CIMRGBColor","values":[3,111,168,100]}\x00\x00', b'{"type":"CIMRGBColor","values":[85,60,143,100]}\x00\x00', b'{"type":"CIMRGBColor","values":[211,47,0,100]}\x00\x00', b'{"type":"CIMRGBColor","values":[234,123,0,100]}\x00\x00', b'{"type":"CIMRGBColor","values":[234,181,0,100]}\x00\x00', b'{"type":"CIMRGBColor","values":[0,116,38,100]}\x00\x00', b'{"type":"CIMRGBColor","values":[157,72,121,100]}\x00\x00', b'{"type":"CIMRGBColor","values":[111,45,61,100]}\x00\x00', b'{"type":"CIMRGBColor","values":[194,60,51,100]}\x00\x00', b'{"type":"CIMRGBColor","values":[226,140,127,100]}\x00\x00', b'{"type":"CIMRGBColor","values":[201,139,58,100]}\x00\x00', b'{"type":"CIMRGBColor","values":[120,157,75,100]}\x00\x00', b'{"type":"CIMRGBColor","values":[89,127,114,100]}\x00\x00', b'{"type":"CIMRGBColor","values":[96,138,143,100]}\x00\x00', b'{"type":"CIMRGBColor","values":[63,116,142,100]}\x00\x00', b'{"type":"CIMRGBColor","values":[8

,ID,Class,Category,Name,colCat,tags,content1,content
0,1,1,,WFRC Blue,"[2, 60, 91, 100]",color;colors;Blue;Logo Brights;WFRC Mountains ...,"b'{""type"":""CIMRGBColor"",""values"":[2,60,91,100]}'","b'{""type"":""CIMRGBColor"",""values"":[2,60,91,100]..."
1,2,1,,Blue,"[3, 111, 168, 100]",color;colors;Blue;Logo Brights;SLawless,"b'{""type"":""CIMRGBColor"",""values"":[3,111,168,10...","b'{""type"":""CIMRGBColor"",""values"":[3,111,168,10..."
2,3,1,,Purple,"[85, 60, 143, 100]",color;colors;Purple;Logo Brights;SLawless,"b'{""type"":""CIMRGBColor"",""values"":[85,60,143,10...","b'{""type"":""CIMRGBColor"",""values"":[85,60,143,10..."
3,4,1,,Red,"[211, 47, 0, 100]",color;colors;Red;Logo Brights;SLawless,"b'{""type"":""CIMRGBColor"",""values"":[211,47,0,100]}'","b'{""type"":""CIMRGBColor"",""values"":[211,47,0,100..."
4,5,1,,Orange,"[234, 123, 0, 100]",color;colors;Orange;Logo Brights;SLawless,"b'{""type"":""CIMRGBColor"",""values"":[234,123,0,10...","b'{""type"":""CIMRGBColor"",""values"":[234,123,0,10..."
5,6,1,,Yellow,"[234, 181, 0, 100]",color;colors;Yellow;Logo Brights;SLawless,"b'{""type"":""CIMRGBColor"",""values"":[234,181,0,10...","b'{""type"":""CIMRGBColor"",""values"":[234,181,0,10..."
6,7,1,,Green,"[0, 116, 38, 100]",color;colors;Green;Logo Brights;SLawless,"b'{""type"":""CIMRGBColor"",""values"":[0,116,38,100]}'","b'{""type"":""CIMRGBColor"",""values"":[0,116,38,100..."
7,8,1,,RTP Fuschia,"[157, 72, 121, 100]",color;colors;Purple;RTP Icons;2019-2050 RTP In...,"b'{""type"":""CIMRGBColor"",""values"":[157,72,121,1...","b'{""type"":""CIMRGBColor"",""values"":[157,72,121,1..."
8,9,1,,RTP Maroon,"[111, 45, 61, 100]",color;colors;Purple;RTP Icons;2019-2050 RTP In...,"b'{""type"":""CIMRGBColor"",""values"":[111,45,61,10...","b'{""type"":""CIMRGBColor"",""values"":[111,45,61,10..."
9,10,1,,RTP Red,"[194, 60, 51, 100]",color;colors;Red;RTP Icons;2019-2050 RTP Infog...,"b'{""type"":""CIMRGBColor"",""values"":[194,60,51,10...","b'{""type"":""CIMRGBColor"",""values"":[194,60,51,10..."


In [178]:
style_colors["key"] = style_colors["Name"].str.lower().str.replace(" ", "_")
style_colors = style_colors.drop(columns=["colCat","content1"])
#style_colors = style_colors.drop(index=0)
style_colors

,ID,Class,Category,Name,tags,content,key
0,1,1,,WFRC Blue,color;colors;Blue;Logo Brights;WFRC Mountains ...,"b'{""type"":""CIMRGBColor"",""values"":[2,60,91,100]...",wfrc_blue
1,2,1,,Blue,color;colors;Blue;Logo Brights;SLawless,"b'{""type"":""CIMRGBColor"",""values"":[3,111,168,10...",blue
2,3,1,,Purple,color;colors;Purple;Logo Brights;SLawless,"b'{""type"":""CIMRGBColor"",""values"":[85,60,143,10...",purple
3,4,1,,Red,color;colors;Red;Logo Brights;SLawless,"b'{""type"":""CIMRGBColor"",""values"":[211,47,0,100...",red
4,5,1,,Orange,color;colors;Orange;Logo Brights;SLawless,"b'{""type"":""CIMRGBColor"",""values"":[234,123,0,10...",orange
5,6,1,,Yellow,color;colors;Yellow;Logo Brights;SLawless,"b'{""type"":""CIMRGBColor"",""values"":[234,181,0,10...",yellow
6,7,1,,Green,color;colors;Green;Logo Brights;SLawless,"b'{""type"":""CIMRGBColor"",""values"":[0,116,38,100...",green
7,8,1,,RTP Fuschia,color;colors;Purple;RTP Icons;2019-2050 RTP In...,"b'{""type"":""CIMRGBColor"",""values"":[157,72,121,1...",rtp_fuschia
8,9,1,,RTP Maroon,color;colors;Purple;RTP Icons;2019-2050 RTP In...,"b'{""type"":""CIMRGBColor"",""values"":[111,45,61,10...",rtp_maroon
9,10,1,,RTP Red,color;colors;Red;RTP Icons;2019-2050 RTP Infog...,"b'{""type"":""CIMRGBColor"",""values"":[194,60,51,10...",rtp_red


In [179]:
conn = eng.raw_connection()
curs = conn.cursor()
#for r in style_colors:
#    print r
#    curs.execute('INSERT INTO ITEMS(ID,CLASS,CATEGORY,NAME,TAGS,KEY) VALUES(?,?,?,?,?,?,?)', r)
#conn.commit()
style_colors.to_sql(name="ITEMS", con=eng, if_exists="replace",index=False)
#eng.execute()
#df = pd.read_sql('select * from ITEMS', eng)
#df

30

In [180]:
#conn.close
insp = sqlalchemy.inspect(eng)
columnst_table= insp.get_columns("items")
for c in columnst_table:
    print(c['name'], c['type'])

#metadata_obj = sqlalchemy.MetaData()
eng.dispose

ID BIGINT
Class BIGINT
Category TEXT
Name TEXT
tags TEXT
content TEXT
key TEXT


<bound method Engine.dispose of Engine(sqlite:///WFRC.stylx)>